# Evolution of Channel Balance

For a given payment stream $(\tau_n, \Delta_n)_{n \geq 1}$, let ```optimal_l_A``` and ```optimal_l_B``` be approximations of Alice and Bob's optimal channel deposits, respectively, and ```optimal_cost_vector``` denote an approximation of the corresponding optimal transaction cost function. These can be computed from the GeneralMethod notebook. The goal of this notebook is to visualize how Bob's balance evolves when the optimal policy is employed over the interval $[0,T]$.

## Inputs

- Cost of an on-chain transaction $C > 0$
- Channel reset cost $D > 0$
- Interest rate $r > 0$
- Time horizon $T > 0$
- Payment stream generating function ```simulate_path```
- Approximation of the optimal channel deposits ```optimal_l_A``` and ```optimal_l_B```
- Approximation of the optimal transaction cost function ```optimal_cost_vector```
- Fine-tuning parameter $n \in \mathbb{N}$
- Second fine-tuning parameter $k \in \mathbb{N}_0$

## Outputs

- Visual representation of how Bob's balance evolves over the interval $[0,T]$ (for a single path of the payment stream)
- The expected cost over $[0, T]$ (by simulating multiple paths of the payment stream)

## Visualizing Bob's Balance

In [54]:
import numpy as np
import matplotlib.pyplot as plt

This function allows us to simulate a single path of the payment stream over $[0,T]$. In all of our examples, $\tau_n$ is the arrival time of the $n^{th}$ point of a Poisson process of rate $\lambda > 0$ and $\Delta$ has a mixture exponential distribution. Therefore, this function only deals with this case.

In [56]:
def sample_mark(p, lam_A, lam_B):
    """
    Generate a random mark Delta_n using the mixture exponential distribution.
    
    Parameters:
    - p: Probability Bob makes a payment. 
    - lam_A: Rate parameter for Alice.
    - lam_B: Rate parameter for Bob.
    
    Returns:
    - A sample from the mixture distribution.
    """
    if np.random.uniform() < p:
        # Bob makes an exponential payment with mean 1/lam_B with probability p
        return np.random.exponential(1 / lam_B)
    else:
        # Alice makes an exponential payment with mean 1/lam_A with probability 1-p
        return -np.random.exponential(1 / lam_A)

def simulate_path(rate_lambda, p, lam_A, lam_B, T):
    """
    Simulate a marked point process path up to time T.
    
    Parameters:
    - rate_lambda: Rate of the Poisson process.
    - p, lam_A, lam_B: Parameters of the mark density.
    - T: Time horizon for the simulation.
    
    Returns:
    - A list of tuples (tau_n, Delta_n).
    """
    # Step 1: Simulate arrival times
    arrival_times = []
    t = 0
    while t < T:
        t += np.random.exponential(1 / rate_lambda)
        if t < T:
            arrival_times.append(t)
    
    # Step 2: Simulate marks (payments)
    marks = [sample_mark(p, lam_A, lam_B) for _ in arrival_times]
    
    # Step 3: Combine
    return list(zip(arrival_times, marks))

Now a payment channel can be modeled by an interval $[0, l_A + l_B]$ where $l_A, l_B > 0$ are the amounts Alice and Bob initially deposit. The state $b \in [0, l_A + l_B]$ represents Bob's balance, and his initial balance is $l_B$. When a payment $\delta \in \mathbb{R}$ is due, then using the first two tables from the GeneralMethod notebook:

- If $b - \delta \notin [0, l_A + l_B]$, then the optimal action is to pay on-chain $(Cn, W)$, which costs an amount $C$, and Bob's balance remains the same $b$.
- If $b - \delta \in [0, l_A + l_B]$, then:
  - If $b \in [0, T_2) \cup (T_3, l_A + l_B]$ and $b - \delta \in [0, T_1) \cup (T_4, l_A+l_B]$, then the optimal action is $(Cl, R)$, which costs an amount $D$, and Bob's new balance is $l_B$.
  - If $b \in [0, T_2) \cup (T_3, l_A + l_B]$ and $b - \delta \in [T_1, T_4]$, then the optimal action is $(Cl, W)$, which costs nothing, and Bob's new balance is $b - \delta$.
  - If $b \in [T_2, T_3]$ and $\delta \in [\delta_{-}(b), \delta_{+}(b)]$, then the optimal action is $(Cl, W)$, which costs nothing, and  Bob's new balance is $b - \delta$.
  - If $b \in [T_2, T_3]$ and $\delta \notin [\delta_{-}(b), \delta_{+}(b)]$, then the optimal action is $(Cn, W)$, which costs an amount $C$, and Bob's balance remains the same $b$.

This *completely* describes how Bob's balance evolves. The thresholds $T_1$, $T_2$, $T_3$ and $T_4$ can be computed using the GeneralMethod notebook, as can the functions $\delta_+$ and $\delta_{-}$. The following code then allows us to visualize how Bob's balance evolves when Alice and Bob behave optimally for a single simulated payment stream path.

In [60]:
def evolve_channel(path, l_A, l_B, T_1, T_2, T_3, T_4, delta_minus_function, delta_plus_function, C, D, n, k):
    """
    Evolve the state of the payment channel over time based on the given rules.
    
    Parameters:
    - path: List of tuples (tau_n, delta_n) representing payment stream.
    - l_A, l_B: Initial deposits of Alice and Bob.
    - T_1, T_2, T_3, T_4: Thresholds for channel state.
    - delta_minus_function, delta_plus_function: Largest amounts Alice and Bob pay on-channel.
    - C, D: Costs for on-chain payment and channel reset.
    - n, k: Fine-tuning parameters.
    
    Returns:
    - List of (time, balance, cost, action) representing the channel state evolution.
    """
    m = (2 ** k) * n
    channel_length = l_A + l_B
    b = l_B  # Bob's initial balance
    evolution = []
    
    for tau, delta in path:
        if b - delta < 0 or b - delta > channel_length:
            # Case one: Outside the channel range
            action = 'Cn, W'
            cost = C
            next_b = b
        else:
            # Case two: Within the channel range
            if b < T_2 + 1/m or b > T_3 - 1/m:
                # Subcases one and two
                if b - delta < T_1 or b - delta > T_4:
                    action = 'Cl, R'
                    cost = D
                    next_b = l_B
                else:
                    action = 'Cl, W'
                    cost = 0
                    next_b = b - delta
            else:
                # Subcases three and four
                if delta_minus_function(b) <= delta <= delta_plus_function(b):
                    action = 'Cl, W'
                    cost = 0
                    next_b = b - delta
                else:
                    action = 'Cn, W'
                    cost = C
                    next_b = b
        
        # Record evolution
        evolution.append((tau, next_b, cost, action))
        b = next_b
    
    return [(0, l_B, 0, 'Cl, W')] + evolution


def visualize_channel_evolution(evolution, payment_stream, l_A, l_B, T_1, T_2, T_3, T_4, T):
    """
    Visualization of channel evolution and payment stream with refined, uniform-width bars.
    
    Parameters:
    - evolution: List of (time, balance, cost, action) representing channel evolution.
    - payment_stream: List of (arrival_time, payment_amount).
    - T: Time horizon.
    """
    # Extract data from evolution and payment_stream
    times, balances, costs, actions = zip(*evolution)
    payment_times, payment_amounts = zip(*payment_stream)
    abs_payment_sizes = np.abs(payment_amounts)
    
    # Setup the figure with two subplots
    fig, (ax1, ax2) = plt.subplots(2, 1, figsize=(14, 8), sharex=True, gridspec_kw={'height_ratios': [3, 1]})
    
    # Plot channel balance evolution (upper plot)
    for i, (tau, balance, cost, action) in enumerate(evolution[1:], start=1):
        prev_time, prev_balance = evolution[i - 1][0], evolution[i - 1][1]
        if action == "Cl, W":
            # Color based on payment direction
            color = "blue" if payment_amounts[i - 1] > 0 else "orange"
            ax1.step([prev_time, tau], [prev_balance, balance], where='post', color=color) 
        elif action == "Cl, R":
            ax1.scatter(tau, balance, color="red", s=40, label="Cl, R (Reset)" if i == 1 else None)
        elif action == "Cn, W":
            ax1.scatter(tau, balance, color="black", s=40, label="Cn, W (On-chain)" if i == 1 else None)
    
    ax1.set_ylabel("Channel Balance")
    ax1.set_xlim(0, T)
    ax1.grid(alpha=0.5)
    ax1.axhline(0, color='black', linestyle='--', linewidth=0.8, label='Channel Boundaries')
    ax1.axhline(l_A + l_B, color='black', linestyle='--', linewidth=0.8)

    if T_1 > 0:
        ax1.axhline(y = T_1, color='r', linestyle='--')
    if T_2 > 0:
        ax1.axhline(y = T_2, color='r', linestyle='--')
    if T_3 < l_A + l_B:
        ax1.axhline(y = T_3, color='r', linestyle='--')
    if T_4 < l_A + l_B:
        ax1.axhline(y = T_4, color='r', linestyle='--')

    # Plot payment stream (lower plot)
    bar_width = T / (len(payment_stream))  # Equal and proportional bar width
    bar_colors = ["blue" if amt > 0 else "orange" for amt in payment_amounts]
    ax2.bar(payment_times, abs_payment_sizes, width=bar_width, color=bar_colors, alpha=0.8, label="Payment Sizes", align='center')
    ax2.set_ylabel("Payment Size")
    ax2.set_ylim(0, max(abs_payment_sizes) * 1.2)
    ax2.set_xlabel("Time")
    ax2.grid(alpha=0.5)

    # Final adjustments
    # plt.suptitle("Channel Balance Evolution", fontsize=14)
    # fig.tight_layout(rect=[0, 0, 1, 0.97])  # Adjust layout to fit the title

## Computing the Expected Cost

The discounted transaction fees accumulated over the interval $[0,T]$ for a single path of the payment stream can be computed using the following function:

In [62]:
def total_discounted_cost(evolution, r):
    """
    Compute the total discounted running cost over the time interval [0, T].
    
    Parameters:
    - evolution: List of (time, balance, cost, action) representing the channel state evolution.
    - r: Discount rate.
    
    Returns:
    - Total discounted cost.
    """
    discounted_cost = 0.0
    
    for tau, _, cost, _ in evolution:
        discounted_cost += np.exp(-r * tau) * cost
    
    return discounted_cost

Repeating this for a large number of paths allows us to compute the expected cost and visualize the cost distribution.

In [64]:
def simulate_multiple_paths(num_paths, rate_lambda, p, lam_A, lam_B, T, l_A, l_B, T_1, T_2, T_3, T_4, 
                            delta_minus_function, delta_plus_function, C, D, r, n, k):
    """
    Simulate multiple payment paths and compute total discounted costs.
    
    Parameters:
    - num_paths: Number of payment paths to simulate.
    - rate_lambda, p, lam_A, lam_B: Parameters for the payment stream.
    - T: Time horizon.
    - l_A, l_B: Initial deposits of Alice and Bob.
    - T_1, T_2, T_3, T_4: Thresholds for channel state.
    - delta_minus, delta_plus: Functions defining subcases (iii) range.
    - C, D: Costs for on-chain payment and channel reset.
    - r: Discount rate.
    - n, k: Fine-tuning parameters.
    
    Returns:
    - List of total discounted costs for each path.
    """
    costs = []
    for _ in range(num_paths):
        # Simulate a payment path
        path = simulate_path(rate_lambda, p, lam_A, lam_B, T)
        
        # Evolve the channel state
        evolution = evolve_channel(path, l_A, l_B, T_1, T_2, T_3, T_4, delta_minus_function, delta_plus_function, C, D, n, k)
        
        # Compute total discounted cost
        total_cost = total_discounted_cost(evolution, r)
        costs.append(total_cost)
    
    return costs

def visualize_cost_distribution(costs, num_paths):
    """
    Visualize the distribution of total discounted costs across multiple paths.
    
    Parameters:
    - costs: List of total discounted costs for each path.
    - num_paths: Number of paths.
    """
    # Histogram of costs
    plt.figure(figsize=(12, 6))
    plt.subplot(1, 2, 1)
    plt.hist(costs, bins=20, color='skyblue', edgecolor='black', alpha=0.7)
    plt.title("Distribution of Discounted Costs")
    plt.xlabel("Discounted Cost")
    plt.ylabel("Frequency")
    
    # Convergence of mean cost
    cumulative_mean = [np.mean(costs[:i+1]) for i in range(len(costs))]
    plt.subplot(1, 2, 2)
    plt.plot(range(1, num_paths + 1), cumulative_mean, color='green', label='Cumulative Mean')
    plt.axhline(np.mean(costs), color='red', linestyle='--', label='Final Mean')
    plt.title("Convergence of Discounted Cost")
    plt.xlabel("Number of Paths")
    plt.ylabel("Mean Total Discounted Cost")
    plt.legend()
    plt.grid(alpha=0.4)
    
    plt.tight_layout()
    plt.show()